<a href="https://colab.research.google.com/github/bushragit/Langchain/blob/main/langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain_groq langchain langchain_core "langchain-chroma>=0.1.2" langchain_community sentence_transformers fastembed

In [ ]:
import getpass
import os
os.environ["GROQ_API_KEY"]=getpass.getpass("Enter Key: ")

In [ ]:
from langchain_groq import ChatGroq
model = ChatGroq(
    model = "llama3-8b-8192",
    temperature = 0,
    max_tokens=None, #if None llm default
    timeout=None,
    max_retries=2
)

In [ ]:
result = model.invoke("translate this sentence in urdu I am Beautiful")
print(result)
print(result.content)

In [ ]:
#now control prompt
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [ ]:
messages=[
    SystemMessage(content="Solve this math problem"),
    HumanMessage(content="what is 2+65?"),
    AIMessage(content="The answer to 2+65 is 67."), #we actually send history to url llm
    HumanMessage(content="what is 2*65?")

]

result = model.invoke(messages)
print(result)
print(result.content)

## Context Length
Context length refers to the amount of previous information or data that a model, such as a language model, can consider or use when generating a response, making predictions, or processing input.It determines how much of the previous conversation, text, or sequence the model can "remember" and use to generate the next word or make predictions.

In [ ]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
messages = [
    ("system","you are a comedian who tells  jokes and you tell maximum 3 jokes"),
    ("human","tell me {count} joke about {topic}")
]

prompt_template = ChatPromptTemplate.from_messages(messages)
prompt = prompt_template.invoke({"topic":"engineers", "count": 3})
print(prompt)
result=model.invoke(prompt)
print(result.content)

In [ ]:
#multi step interaction using LLM
from langchain.schema.output_parser import StrOutputParser

In [ ]:
#output of promptemplate is prompt
chain = prompt_template | model | StrOutputParser()
chain.invoke({"topic":"engineers", "count": 3})

### CHAIN:

In [ ]:
from langchain.schema.runnable import RunnableBranch, RunnableLambda

In [ ]:
positive_feedback_template = ChatPromptTemplate.from_messages([
    ("system", "you are a helpfull assistant"),
    ("human",
     "generate a thank you email for the positive feedback: {feedback}."),
])

neutral_feedback_template = ChatPromptTemplate.from_messages([
    ("system", "you are a helpfull assistant"),
    ("human",
     "generate a request for more details on this neutral feedback from a user: {feedback}."),
])

negative_feedback_template = ChatPromptTemplate.from_messages([
    ("system", "you are a helpfull assistant"),
    ("human",
     "generate a apology note for user for this negative feedback : {feedback}."),
])

escalate_feedback_template = ChatPromptTemplate.from_messages([
    ("system", "you are a helpfull assistant"),
    ("human",
     "generate a msg to escalate this feedback to a human agent: {feedback}."),
])

classification_feedback_template = ChatPromptTemplate.from_messages([
    ("system", "you are a helpfull assistant"),
    ("human",
     "classify the sentiment of this  feedback as positive, negative, neutral or escalate and give response as one word in lower case: {feedback}."),
])

classification_chain=(
    classification_feedback_template|model|StrOutputParser()
)


branches = RunnableBranch(
    (
        lambda x: "positive" in x,
        positive_feedback_template|model|StrOutputParser()

    ),
    (
        lambda x: "negative" in x,
        negative_feedback_template|model|StrOutputParser()
    ),
    (
        lambda x: "neutral" in x,
        neutral_feedback_template|model|StrOutputParser()
    ),
     escalate_feedback_template|model|StrOutputParser()
)

#when input enter what to do step by step
#define chain of classification


#final chain
chain = classification_chain | branches

#result
review="the movie is great"
result=chain.invoke({"feedback": review})
print(result)